In [1]:
import lxml.html
from pathlib import Path
import pickle
import re
import time
from tqdm import tqdm

from projectmir.xmldocument import XMLDocument
from projectmir.functions import find_identifier_definition

In [2]:
p_path = './data/test_latexml/'
p = Path(p_path)
x = list(p.glob('*.html'))
documents_IDs = [x_.name[:-5] for x_ in x]

doc = [[]]*len(documents_IDs)
for i, documents_ID in tqdm(enumerate(documents_IDs)):
    documents_path = p_path + documents_ID + '.html'
    doc_ = find_identifier_definition(documents_path)
    doc[i] = doc_

save_filename = 'doc' + time.strftime('_%d%b%Y_%H%M%S') + '.pkl'
with open(save_filename, 'wb') as f:
    pickle.dump(doc, f)


0it [00:00, ?it/s]elapsed time : 0.0000229 seconds ---
process document
elapsed time : 0.0029061 seconds ---
Number of math components is 50
elapsed time : 0.0467198 seconds ---
elapsed time : 0.0914931 seconds ---
Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 600000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-dcc99becbf204b4d.props -preload tokenize,ssplit,pos
elapsed time : 3.0090470 seconds ---
Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 600000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-dabd9e10d16f464d.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
1it [00:43, 43.80s/it]extract_noun_phrase

In [3]:
# load_filename = 'doc.pkl'
load_filename = save_filename
with open(load_filename, 'rb') as f:
    doc = pickle.load(f)
print(doc[0].path)
print(doc[0].identifiers)

./data/test_latexml/test_sample.html
['w_i', 'T_i', 'Q', 'T', 'w', 'V', 'ρ', 'C', 'U_int', 'H', '\\overset{^}{H}', 'd', 't', 'T_ref', 'Δ', 'N_F']


In [4]:
load_filename = 'doc.pkl'
# load_filename = save_filename
with open(load_filename, 'rb') as f:
    doc = pickle.load(f)
print(doc[0].path)
print(doc[0].identifiers)

./data/test_latexml/test_sample.html
['w_i', 'T_i', 'Q', 'T', 'w', 'V', 'ρ', 'C', 'U_int', 'H', 'd', 't', 'T_ref', 'Δ', 'N_F']


In [5]:
tree = lxml.html.parse(doc[0].path)

In [6]:
html = tree.getroot()

In [64]:
x_list = html.cssselect('mi')
x_list[0]

<Element mi at 0x120d7c360>

In [13]:
def is_identifier(math_component):
    is_mi = (math_component.tag == 'mi')
    is_math_component_len_1 = (len(math_component.text_content()) == 1)
    is_italic = (math_component.get('mathvariant') == 'italic')
    return is_mi and (is_math_component_len_1 or is_italic)

def extract_ml_component(html_cssselect_math, mltag, identifiers, reg_string_list):
    identifiers_ = list(identifiers)
    reg_string_list_ = list(reg_string_list)
    for html_math_mltag in html_cssselect_math.cssselect(mltag):
        if mltag == 'mi':
            math_txt = html_math_mltag.text_content()
            identifier_candidate = html_math_mltag
        else:
            html_math_mltag_component = [
                x for x in html_math_mltag.iterchildren()]
            identifier_candidate = html_math_mltag_component[0]
            math_txt = [x.text_content()
                        for x in html_math_mltag_component]
            if mltag == 'msubsup':
                math_txt = math_txt[0] + '_' + \
                    math_txt[1] + '^' + math_txt[2]
            elif mltag == 'msub':
                math_txt = math_txt[0] + '_' + math_txt[1]
            elif mltag == 'msup':
                math_txt = math_txt[0] + '^' + math_txt[1]
            elif mltag == 'mover':
                math_txt = r'\overset{' + \
                    math_txt[1] + '}{' + math_txt[0] + '}'
            elif mltag == 'munder':
                math_txt = r'\underset{' + \
                    math_txt[1] + '}{' + math_txt[0] + '}'
            elif mltag == 'munderover':
                math_txt = r'\overset{' + math_txt[2] + '}{' + \
                    r'\underset{' + math_txt[1] + '}{' + math_txt[0] + '}}'

        if is_identifier(identifier_candidate) and (
                math_txt not in identifiers_):
            identifiers_.append(math_txt)
        html_math_mltag.drop_tree()  # this is needed for extracting w_i without w and i.
        reg_string = lxml.html.tostring(
            html_math_mltag, encoding='unicode')
        if is_identifier(identifier_candidate):
            reg_string_list_.append(
                (math_txt, reg_string, 'MATH{:04d}'.format(
                    identifiers_.index(math_txt))))
        else:
            reg_string_list_.append(
                (math_txt, reg_string, math_txt))
    return identifiers_, reg_string_list_

In [147]:
# x_text = """
# <p class="ltx_p">Next, we show how the general energy balance in Eq. 2-10 can be simplified for this particular example.For a pure liquid at low or moderate pressures, the internal energy is approximately equal to the enthalpy, <math id="S1.SS1.p2.m1" class="ltx_Math" alttext="U_{\rm int}\approx H" display="inline"><mrow><msub><mi>U</mi><mi>int</mi></msub><mo>≈</mo><mi>H</mi></mrow></math>, and <math id="S1.SS1.p2.m2" class="ltx_Math" alttext="H" display="inline"><mi>H</mi></math> depends only on temperature (Sandler, 2006).Consequently, in the subsequent development, we assume that <math id="S1.SS1.p2.m3" class="ltx_Math" alttext="U_{\rm int}=H" display="inline"><mrow><msub><mi>U</mi><mi>int</mi></msub><mo>=</mo><mi>H</mi></mrow></math> and <math id="S1.SS1.p2.m4" class="ltx_Math" alttext="\hat{U}_{\rm int}=\hat{H}" display="inline"><mrow><msub><mover accent="true"><mi>U</mi><mo stretchy="false">^</mo></mover><mi>int</mi></msub><mo>=</mo><mover accent="true"><mi>H</mi><mo stretchy="false">^</mo></mover></mrow></math> where the caret (^) means per unit mass.As shown in Appendix C, a differential change in temperature <math id="S1.SS1.p2.m5" class="ltx_Math" alttext="dT" display="inline"><mrow><mi>d</mi><mo>⁢</mo><mi>T</mi></mrow></math>, produces a corresponding change in the internal energy per unit mass, <math id="S1.SS1.p2.m6" class="ltx_Math" alttext="d\hat{U}_{\rm int}" display="inline"><mrow><mi>d</mi><mo>⁢</mo><msub><mover accent="true"><mi>U</mi><mo stretchy="false">^</mo></mover><mi>int</mi></msub></mrow></math>,</p>
# """
x_text = """
<p class="ltx_p">Consequently, in the subsequent development, we assume that <math id="S1.SS1.p2.m3" class="ltx_Math" alttext="U_{\rm int}=H" display="inline"><mrow><msub><mi>U</mi><mi>int</mi></msub><mo>=</mo><mi>H</mi></mrow></math> and <math id="S1.SS1.p2.m4" class="ltx_Math" alttext="\hat{U}_{\rm int}=\hat{H}" display="inline"><mrow><msub><mover accent="true"><mi>U</mi><mo stretchy="false">^</mo></mover><mi>int</mi></msub><mo>=</mo><mover accent="true"><mi>H</mi><mo stretchy="false">^</mo></mover></mrow></math> where the caret (^) means per unit mass.</p>
"""

In [151]:
import copy

In [154]:
i, r = [], []
x_tree = lxml.html.fromstring(x_text)
print(x_tree.text_content())
for x_math in copy.deepcopy(x_tree).cssselect('math'):
    print(extract_ml_component(x_math, 'mover', i, r))
print(x_tree.text_content())

# for html_math_mltag in html_cssselect_math.cssselect(mltag):
#     if mltag == 'mi':
#         math_txt = html_math_mltag.text_content()
#         identifier_candidate = html_math_mltag
#     else:
#         html_math_mltag_component = [
#             x for x in html_math_mltag.iterchildren()]
#         identifier_candidate = html_math_mltag_component[0]
#         math_txt = [x.text_content()
#                     for x in html_math_mltag_component]
#         if mltag == 'msubsup':
#             math_txt = math_txt[0] + '_' + \
#                 math_txt[1] + '^' + math_txt[2]
#         elif mltag == 'msub':
#             math_txt = math_txt[0] + '_' + math_txt[1]
#         elif mltag == 'msup':
#             math_txt = math_txt[0] + '^' + math_txt[1]
#         elif mltag == 'mover':
#             math_txt = r'\overset{' + \
#                 math_txt[1] + '}{' + math_txt[0] + '}'
#         elif mltag == 'munder':
#             math_txt = r'\underset{' + \
#                 math_txt[1] + '}{' + math_txt[0] + '}'
#         elif mltag == 'munderover':
#             math_txt = r'\overset{' + math_txt[2] + '}{' + \
#                 r'\underset{' + math_txt[1] + '}{' + math_txt[0] + '}}'

Consequently, in the subsequent development, we assume that Uint=H and U^int=H^ where the caret (^) means per unit mass.
([], [])
(['\\overset{^}{U}', '\\overset{^}{H}'], [('\\overset{^}{U}', '<mover accent="true"><mi>U</mi><mo stretchy="false">^</mo></mover>', 'MATH0000'), ('\\overset{^}{H}', '<mover accent="true"><mi>H</mi><mo stretchy="false">^</mo></mover>', 'MATH0001')])
Consequently, in the subsequent development, we assume that Uint=H and U^int=H^ where the caret (^) means per unit mass.


In [164]:
print([i for i in copy.deepcopy(x_tree).cssselect('math')[1].cssselect('msub')[0].iterchildren()])

[<Element mover at 0x128b42d10>, <Element mi at 0x128b42c20>]


In [125]:
print(f'{i=}')
print(f'{r=}')

i=[]
r=[]


In [32]:
x_list[13].text_content()

<bound method HtmlMixin.text_content of <Element math at 0x12ba98040>>